In [10]:
import sys
import os

sys.path.append(os.path.abspath('..'))

from torch.utils.data import DataLoader

import torch.optim as optim
import torch
import torch.nn as nn

from tqdm.notebook import tqdm

from CustomDatasets import SCAEDataset
from Models import StackedConvAutoencoder, ArtifactCNN
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:
train_dataset = SCAEDataset(root_dir="../data/train")
val_dataset = SCAEDataset(root_dir="../data/val")
test_dataset = SCAEDataset(root_dir="../data/test")

train_loader = DataLoader(train_dataset, batch_size=32, num_workers=4,shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, num_workers=4,shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=4,shuffle=True, pin_memory=True)

In [11]:
SCAEModel = StackedConvAutoencoder()

#Load the best model while training the AE
SCAEModel.load_state_dict(torch.load('../models/SCAE_best.pth'))
SCAEModel.to(device)
SCAEModel.eval()

StackedConvAutoencoder(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (unpool): MaxUnpool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
  (conv2): ConvTranspose2d(16, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
)

In [12]:
num_classes = 2
model = ArtifactCNN(num_classes=num_classes)
print(model)

ArtifactCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


In [13]:
## Verifying sizes of each layer outputs
dummy_input = torch.randn(32, 1, 64, 64)

# Function to calculate the size of each layer's output
def calculate_layer_sizes(model, dummy_input):
    layer_sizes = []
    x = dummy_input
    for layer in model.children():
        x = layer(x)
        if isinstance(layer, nn.Flatten):
            layer_sizes.append(x.shape[1])  # After flattening, x is a 1D tensor
        else:
            layer_sizes.append(x.size())
    return layer_sizes

layer_sizes = calculate_layer_sizes(model, dummy_input)
layer_sizes

[torch.Size([32, 16, 32, 32]),
 torch.Size([32, 16, 16, 16]),
 torch.Size([32, 32, 16, 16]),
 8192,
 torch.Size([32, 128]),
 torch.Size([32, 128]),
 torch.Size([32, 2])]

In [17]:
# Train the CNN
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.to(device)

num_epochs = 100  # Number of training epochs
best_val_loss = float('inf')  # Track the best validation loss

# Training and Validation
for epoch in tqdm(range(num_epochs)):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device).float(), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        with torch.no_grad():
            inputs_recon = SCAEModel(inputs)
        
        
        # Forward + backward + optimize
        outputs = model(inputs_recon)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    # Print average training loss per epoch
    avg_train_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}')
    
    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device).float(), labels.to(device)
            inputs_recon = SCAEModel(inputs)

            outputs = model(inputs_recon)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    avg_val_loss = val_loss / len(val_loader)
    print(f'Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}')
    
    # Save the model if validation loss has decreased
    if avg_val_loss < best_val_loss:
        print(f'Validation Loss Decreased({best_val_loss:.4f}--->{avg_val_loss:.4f}) \t Saving The Model')
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), '../models/CNN_best_model.pth')

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1, Train Loss: 0.8132
Epoch 1, Validation Loss: 0.8133
Validation Loss Decreased(inf--->0.8133) 	 Saving The Model
Epoch 2, Train Loss: 0.8133
Epoch 2, Validation Loss: 0.8127
Validation Loss Decreased(0.8133--->0.8127) 	 Saving The Model
Epoch 3, Train Loss: 0.8133
Epoch 3, Validation Loss: 0.8142
Epoch 4, Train Loss: 0.8132
Epoch 4, Validation Loss: 0.8133
Epoch 5, Train Loss: 0.8133
Epoch 5, Validation Loss: 0.8135
Epoch 6, Train Loss: 0.8133
Epoch 6, Validation Loss: 0.8127
Epoch 7, Train Loss: 0.8132
Epoch 7, Validation Loss: 0.8129
Epoch 8, Train Loss: 0.8134
Epoch 8, Validation Loss: 0.8129
Epoch 9, Train Loss: 0.8133
Epoch 9, Validation Loss: 0.8138
Epoch 10, Train Loss: 0.8134
Epoch 10, Validation Loss: 0.8133
Epoch 11, Train Loss: 0.8134
Epoch 11, Validation Loss: 0.8135
Epoch 12, Train Loss: 0.8134
Epoch 12, Validation Loss: 0.8135
Epoch 13, Train Loss: 0.8132
Epoch 13, Validation Loss: 0.8136
Epoch 14, Train Loss: 0.8133
Epoch 14, Validation Loss: 0.8142
Epoch 15, Tra